# Imports

In [1]:
import config
import utility

import cv2
from keras.applications.inception_v3 import InceptionV3, preprocess_input, decode_predictions
from keras.preprocessing import image, text
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Input
from keras import backend as K
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping, ModelCheckpoint, Callback
from keras.optimizers import SGD
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle as pkl
from sklearn.utils import class_weight
from tqdm import tqdm_notebook as tqdm

Using TensorFlow backend.


In [2]:
BATCH_SIZE=32

# Check Devices

In [3]:
utility.check_devices()

Device List
	 CPU
		name: /device:CPU:0
		memory_limit: 268435456
Device List
	 GPU
		name: /device:GPU:0
		memory_limit: 3868721152


# Import Data

In [4]:
df_train = pkl.load(open('./data/df_train.dump.pkl', 'rb'))
df_valid = pkl.load(open('./data/df_valid.dump.pkl', 'rb'))
task_id_to_object_map = pkl.load(open('./data/task_id_to_object_map.dump.pkl', 'rb'))

In [5]:
train_labels = {}
for path, taskId in zip(df_train.imageId.tolist(), df_train.taskId.tolist()):
    train_labels[path.split('./train_images/')[1]] = task_id_to_object_map[taskId]
valid_labels = {}
for path, taskId in zip(df_valid.imageId.tolist(), df_valid.taskId.tolist()):
    valid_labels[path.split('./valid_images/')[1]] = task_id_to_object_map[taskId]

In [6]:
resized_train = os.listdir(config.RESIZED_TRAIN_DIR)
resized_valid = os.listdir(config.RESIZED_VALID_DIR)

In [7]:
# del X_train, X_valid
X_train = np.empty(shape=(len(resized_train), config.MAX_PIXEL, config.MAX_PIXEL, 3), dtype=np.uint8)
labels_train = []
X_valid = np.empty(shape=(len(resized_valid), config.MAX_PIXEL, config.MAX_PIXEL, 3), dtype=np.uint8)
labels_valid = []

In [8]:
for idx, _ in tqdm(enumerate(resized_train), total=len(resized_train)):
    img = cv2.imread(config.RESIZED_TRAIN_DIR + "/" + _)
    h, w, c = img.shape
    X_train[idx, :h, :w] = img
    labels_train.append(train_labels[_])

In [9]:
for idx, _ in tqdm(enumerate(resized_valid), total=len(resized_valid)):
    img = cv2.imread(config.RESIZED_VALID_DIR + "/" + _)
    h, w, c = img.shape
    X_valid[idx, :h, :w] = img
    labels_valid.append(valid_labels[_])

In [10]:
label_tokenizer = text.Tokenizer()
label_tokenizer.fit_on_texts(labels_valid)

In [11]:
y_train = label_tokenizer.texts_to_sequences(labels_train)
y_valid = label_tokenizer.texts_to_sequences(labels_valid)

In [12]:
y_train = [_[0] for _ in y_train]
y_valid = [_[0] for _ in y_valid]
cls_wgt = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)
cls_wgt = np.array([0] + cls_wgt.tolist())
cls_wgt = {idx: value for idx, value in enumerate(cls_wgt)}

In [13]:
cls_wgt

{0: 0.0,
 1: 0.8259746359793331,
 2: 0.8457579838399384,
 3: 1.1644153092305656,
 4: 1.3366524779568258}

In [14]:
y_train = to_categorical(np.array(y_train))
y_valid = to_categorical(np.array(y_valid))

In [15]:
print("Shapes:")
print("X_train:", X_train.shape)
print("y_train:", y_train.shape)
print("X_valid:", X_valid.shape)
print("y_valid:", y_valid.shape)

Shapes:
X_train: (35170, 128, 128, 3)
y_train: (35170, 5)
X_valid: (7074, 128, 128, 3)
y_valid: (7074, 5)


In [ ]:
train_generator = image.ImageDataGenerator(featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=False
)

valid_generator = image.ImageDataGenerator()

In [ ]:
train_generator.fit(X_train)

In [16]:
# reference: https://keras.io/applications/
input_tensor = Input(shape=(config.MAX_PIXEL, config.MAX_PIXEL, 3))
base_model = InceptionV3(input_tensor=input_tensor, weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(5, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)
for layer in base_model.layers:
    layer.trainable = False
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

In [17]:
early_stopping =EarlyStopping(monitor='val_loss', patience=3)
bst_model_path = 'model.h5'
model_checkpoint = ModelCheckpoint(bst_model_path, save_best_only=True, save_weights_only=True)

In [19]:
model.fit(
    X_train, y_train,
    batch_size=BATCH_SIZE,
    epochs=5, 
    validation_data=(X_valid, y_valid),
    class_weight=cls_wgt,
    callbacks=[early_stopping, model_checkpoint],
    shuffle=True)

Train on 35170 samples, validate on 7074 samples
Epoch 1/5
35170/35170 [==============================] - 199s 6ms/step - loss: 1.3682 - acc: 0.3400 - val_loss: 10.5873 - val_acc: 0.2537
Epoch 2/5
35170/35170 [==============================] - 186s 5ms/step - loss: 1.2027 - acc: 0.4732 - val_loss: 9.9883 - val_acc: 0.2909
Epoch 3/5
35170/35170 [==============================] - 185s 5ms/step - loss: 1.1169 - acc: 0.5295 - val_loss: 10.7474 - val_acc: 0.2703
Epoch 4/5
35170/35170 [==============================] - 185s 5ms/step - loss: 1.0633 - acc: 0.5598 - val_loss: 10.7055 - val_acc: 0.2747
Epoch 5/5
35170/35170 [==============================] - 185s 5ms/step - loss: 1.0319 - acc: 0.5721 - val_loss: 10.9005 - val_acc: 0.2682


In [ ]:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

In [ ]:
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True

In [ ]:
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy')
model.fit_generator(
    train_generator.flow(X_train, y_train, batch_size=BATCH_SIZE, shuffle=True), 
    steps_per_epoch=len(X_train)/BATCH_SIZE, 
    epochs=50, 
    validation_data=valid_generator.flow(X_valid, y_valid),
    class_weight=cls_wgt,
    callbacks=[early_stopping, model_checkpoint],
    shuffle=True)

In [ ]:
y_valid_hat = model.predict(X_valid)

In [ ]:
y_valid_hat.argmax(axis=1)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_valid_hat.argmax(axis=1), y_valid.argmax(axis=1))

In [ ]:
img_path = './resized_train/101.jpg'
img = image.load_img(img_path, target_size=(100, 100))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

features = model.predict(x)

In [ ]:
label_tokenizer.word_index

In [ ]:
features

In [ ]:
label_tokenizer.word_index

In [ ]:
np.bincount(y_train)

In [ ]:
class_weights = np.bincount(y_train.argmax(axis=1))
class_weights/max(class_weights)

In [ ]:
class_weights = np.bincount(y_valid.argmax(axis=1))
class_weights/max(class_weights)